This notebook is used to get patient-level scores with the counting method in the paper 'Deep learning can predict
microsatellite instability directly from histology in gastrointestinal cancer'

In [1]:
import openpyxl
from sklearn.metrics import roc_auc_score
import scipy
import math
import os
import numpy as np

In [2]:
def searchFile(key,startPath = '.'):
    if not os.path.isdir(startPath):
        raise ValueError
    l= [os.path.join(startPath,x) for x in os.listdir(startPath)]  #列出所有文件的绝对路径
    #listdir出来的相对路径 不能用于 isfile  abspath只能用在当前目录
    filelist=[x for x in l if os.path.isfile(x) if key in os.path.splitext(os.path.basename(x))[0]] #文件
    #只查找文件名中  不包括后缀 文件路径
    if not hasattr(searchFile,'basePath'):#把函数当成类 添加属性
        searchFile.basePath=startPath #只有第一次调用才会赋值给basePath
    outmap = map(lambda x:os.path.relpath(x,searchFile.basePath),filelist) #转换成相对于初始路径的相对路径

    outlist = list(outmap) 

    dirlist= [x for x in l if os.path.isdir(x)]  #目录
    for dir in dirlist:
        outlist = outlist + searchFile(key,dir)
 
    return outlist

In [4]:
from openpyxl import Workbook
wb = Workbook()
sheet = wb.active
sheet.title = "feature_MSI"

wb2 = Workbook()
sheet2 = wb2.active
sheet2.title = "num_MSI"

info = openpyxl.load_workbook('D:/dataset_Huo_1/MSIMUT_validation.xlsx')
inf = info.active

# num_info = openpyxl.load_workbook('D:/dataset_Huo_1/number/image_num_test_MSI.xlsx')
# num_inf = num_info.active

patch_info = openpyxl.load_workbook('D:/dataset_Huo_1/patch_info/resnet18/MSI_validation_patch_info.xlsx')
patch_inf = patch_info.active
vote1 = []
right_num = 0
for i in range(0,11):
    patient_name = inf.cell(row=1, column=(i+1)).value
    print(i,"patient id:",patient_name)
    file_names = searchFile(patient_name,startPath = 'D:/dataset_Huo_1/CRC_DX/validation/MSIMUT/')
    image_num = len(file_names)
    # image_num = num_inf.cell(i+1,1).value
    sheet2["A%d" % (i+1)].value = image_num
    print("numbers of images of this patient",image_num)
    patient_patch = []
    for j in range(image_num):
        patient_patch.append(patch_inf.cell(row = (j + 1), column = (i + 1)).value)

    counts,bins = np.histogram(patient_patch,bins = 10,range = (0,1))
    counts_norm = counts / image_num
    for k in range(10):
        sheet.cell(i+1,k+1).value = counts_norm[k]
    # for t in range(20):
    #     counts_norm[t] = counts_norm[t] * decay_factor[t]
    # mss = sum(counts_norm[0:10])
    # msi = sum(counts_norm[10:20])

    # vote_buf = msi / (mss + msi)
    # vote1.append(vote_buf)
    # if vote_buf >= 0.5:
    #         right_num = right_num + 1

    # print(i, "vote_MSI_decay:",vote1[i],"mean vote after decay:",sum(vote1)/(i+1))
    print("\n")
# print('total number: 26','right number:',right_num) 
wb.save('D:/dataset_Huo_1/feature_MSI_10.xlsx')
# wb2.save('D:/dataset_Huo_1/number/image_num_train_MSI.xlsx')


0 patient id: TCGA-A6-5665
numbers of images of this patient 439


1 patient id: TCGA-AA-3864
numbers of images of this patient 273


2 patient id: TCGA-AD-6895
numbers of images of this patient 168


3 patient id: TCGA-CK-5913
numbers of images of this patient 1932


4 patient id: TCGA-CK-5916
numbers of images of this patient 1205


5 patient id: TCGA-CM-5861
numbers of images of this patient 1077


6 patient id: TCGA-CM-6171
numbers of images of this patient 4305


7 patient id: TCGA-F4-6703
numbers of images of this patient 1184


8 patient id: TCGA-F4-6856
numbers of images of this patient 370


9 patient id: TCGA-NH-A5IV
numbers of images of this patient 2888


10 patient id: TCGA-EI-6507
numbers of images of this patient 271




In [5]:
wb = Workbook()
sheet = wb.active
sheet.title = "feature_MSS"

wb2 = Workbook()
sheet2 = wb2.active
sheet2.title = "num_MSS"

# num_info = openpyxl.load_workbook('D:/dataset_Huo_1/number/image_num_test_MSS.xlsx')
# num_inf = num_info.active

info = openpyxl.load_workbook('D:/dataset_Huo_1/MSS_validation.xlsx')
inf = info.active

patch_info = openpyxl.load_workbook('D:/dataset_Huo_1/patch_info/resnet18/MSS_validation_patch_info.xlsx')
patch_inf = patch_info.active
vote2 = []
right_num = 0
for i in range(0,61):
    patient_name = inf.cell(row=1, column=(i+1)).value
    print(i,"patient id:",patient_name)
    # image_num = num_inf.cell(i+1,1).value
    file_names = searchFile(patient_name,startPath = 'D:/dataset_Huo_1/CRC_DX/validation/MSS/')
    image_num = len(file_names)
    sheet2["A%d" % (i+1)].value = image_num
    print("numbers of images of this patient",image_num)
    patient_patch = []
    for j in range(image_num):
        patient_patch.append(patch_inf.cell(row = (j + 1), column = (i + 1)).value)

    counts,bins = np.histogram(patient_patch,bins = 10,range = (0,1))
    counts_norm = counts / image_num
    for k in range(10):
        sheet.cell(i+1,k+1).value = counts_norm[k]
    # for t in range(20):
    #     counts_norm[t] = counts_norm[t] * decay_factor[t]
    # mss = sum(counts_norm[0:10])
    # msi = sum(counts_norm[10:20])

    # vote_buf = msi / (mss + msi)
    # vote2.append(vote_buf)
    # if vote_buf < 0.5:
    #         right_num = right_num + 1

    # print(i, "vote_MSS_decay:",vote2[i],"mean vote after decay:",sum(vote2)/(i+1))
    print("\n")
print('total number: 160','right number:',right_num) 
wb.save('D:/dataset_Huo_1/feature_MSS_10.xlsx')
# wb2.save('D:/dataset_Huo_1/number/image_num_train_MSS.xlsx')


0 patient id: TCGA-A6-4107
numbers of images of this patient 103


1 patient id: TCGA-A6-6651
numbers of images of this patient 73


2 patient id: TCGA-A6-A567
numbers of images of this patient 216


3 patient id: TCGA-A6-A56B
numbers of images of this patient 274


4 patient id: TCGA-AA-3666
numbers of images of this patient 169


5 patient id: TCGA-AA-3673
numbers of images of this patient 43


6 patient id: TCGA-AA-3696
numbers of images of this patient 62


7 patient id: TCGA-AA-3812
numbers of images of this patient 131


8 patient id: TCGA-AA-3842
numbers of images of this patient 71


9 patient id: TCGA-AA-3984
numbers of images of this patient 109


10 patient id: TCGA-AA-3989
numbers of images of this patient 94


11 patient id: TCGA-AA-3994
numbers of images of this patient 52


12 patient id: TCGA-AY-4070
numbers of images of this patient 54


13 patient id: TCGA-AZ-6605
numbers of images of this patient 312


14 patient id: TCGA-CA-5796
numbers of images of this patient 96


In [126]:
true_label = []
#MSIMUT
for i in range(26):
    true_label.append(1)
#MSS
for j in range(74):
    true_label.append(0)
pred = np.hstack((vote1,vote2))
auc_score = roc_auc_score(true_label,pred)
print(auc_score)


0.7858627858627859
